In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import load_model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
import os

In [ ]:
# all class names...
dir_list = sorted(os.listdir('../input/plant-seedlings-classification/train'))
class_names_lst = [dir_name for dir_name in dir_list]
class_names_lst

In [ ]:
test_list = sorted(os.listdir('../input/plant-seedlings-classification/test'))

In [ ]:
# load train and test data
image_generator = ImageDataGenerator(
    rescale = 1. / 255,
    horizontal_flip = True,
    vertical_flip = False,
    rotation_range = 25,
    zoom_range=0.5,
    validation_split=0.15
)
train_data = image_generator.flow_from_directory(
    '../input/plant-seedlings-classification/train',
    target_size = (299, 299),
    color_mode = 'rgb',
    classes = class_names_lst,
    class_mode = 'categorical',
    batch_size = 32,
    subset = 'training'
)
valid_data = image_generator.flow_from_directory(
    '../input/plant-seedlings-classification/train',
    target_size = (299, 299),
    color_mode = 'rgb',
    classes = class_names_lst,
    class_mode = 'categorical',
    batch_size = 32,
    subset = 'validation'
)

In [ ]:
test_generator = ImageDataGenerator(
    rescale = 1. / 255,
)
test_data = test_generator.flow_from_directory(
    '../input/plant-seedlings-classification/',
    target_size = (299, 299),
    classes = ['test'],
    class_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 1,
    shuffle = False
)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(299, 299, 3),
                                               include_top=False,
                                               weights="imagenet")
base_model.trainable = False

In [ ]:
early_stop=EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
'''
layers.Conv2D(filters = 32,
                  kernel_size = 3,
                  padding = 'same',
                  activation = 'relu'),
    layers.MaxPool2D(),
    layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'),
    layers.MaxPool2D(),
    layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'),
    layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'),
    layers.MaxPool2D(),
'''

In [ ]:
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(12, activation = 'softmax')
])
model.compile(
    optimizer = 'Adam',
    loss = 'categorical_crossentropy',
    metrics = 'categorical_accuracy'
)
model.summary()

In [ ]:

model.fit(
    train_data,
    epochs = 50,
    batch_size = 32,
    validation_data = valid_data,
    callbacks = [early_stop]
)

In [ ]:
y_hat = model.predict(test_data)

In [ ]:
y_hat

In [ ]:
y_hat.shape

In [ ]:
y_hat_lst = []
for idx in range(y_hat.shape[0]):
    y_hat_lst.append(np.argmax(y_hat[idx]))

In [ ]:
y_hat_cat = []
for val in y_hat_lst:
    y_hat_cat.append(class_names_lst[val])

In [ ]:
submission_df = pd.DataFrame({
    'file': test_list,
    'species': y_hat_cat 
})

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv')